<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Simple-SMA2-(long/short)-strategy" data-toc-modified-id="Simple-SMA2-(long/short)-strategy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Simple SMA2 (long/short) strategy</a></span></li><li><span><a href="#Simple-SMA-4-(long/short)-strategy" data-toc-modified-id="Simple-SMA-4-(long/short)-strategy-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simple SMA 4 (long/short) strategy</a></span></li><li><span><a href="#SMA-2-with-tp-and-sl" data-toc-modified-id="SMA-2-with-tp-and-sl-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SMA 2 with tp and sl</a></span></li><li><span><a href="#SMA-4-with-tp-and-sl" data-toc-modified-id="SMA-4-with-tp-and-sl-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>SMA 4 with tp and sl</a></span></li></ul></div>

In [2]:
from backtesting import Backtest, Strategy
from backtesting.backtesting import Position, Trade, Order
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
from backtesting.test import SMA

from typing import Tuple, Union
from pandas import Series

import pandas_datareader.data as dtr
import requests_cache
import datetime
expire_after = datetime.timedelta(days = 7)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite',
                                       expire_after=expire_after)

tickers = ["LNTA.ME", "MTSS.ME","GAZP.ME","CHMF.ME","SBER.ME"]

df = pd.DataFrame()
for ticker in tickers:
    try:
        df_ = dtr.DataReader(ticker, data_source="yahoo" , session=session, retry_count=1)
        df_.insert(0, "ticker", ticker)
    except:
        print(f"ticker '{ticker}' failed")
        pass
    df = df.append(df_)

GAZP = df[df.ticker == "GAZP.ME"]
GAZP.head()

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

,ticker,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
2015-12-23,GAZP.ME,135.300003,133.350006,133.550003,135.300003,26760910.0,135.300003
2015-12-24,GAZP.ME,138.080002,134.449997,135.850006,136.000000,36828650.0,136.000000
2015-12-25,GAZP.ME,136.490005,134.029999,136.490005,134.479996,14866310.0,134.479996
2015-12-28,GAZP.ME,135.399994,133.759995,134.350006,134.970001,11795110.0,134.970001
2015-12-29,GAZP.ME,137.550003,134.740005,135.259995,137.490005,21516980.0,137.490005


# Simple SMA2 (long/short) strategy

Strategies are constructed by inheriting from `Strategy`.

Once Strategy is initiated the following methods are available:

`print(dir_(Strategy))`

```
['I', '_abc_impl', '_check_params', 'buy', 'closed_trades', 'data', 'equity', 'init', 'next', 'orders', 'position', 'sell', 'trades']
```
 - `self.next`: Main strategy runtime method, called as each new `Strategy.data` instance (row; full candlestick bar) becomes available. This is the main method where strategy decisions upon data precomputed in `Strategy.init()` take place. If you extend composable strategies from `backtesting.lib`, make sure to call: **`super.next()`**
 - `self.position`: instance of `Position`
 - `self.data` holds roughly data passed to Strategy constructor
 - `self.equity` equity position
 - `self.orders` list of pending orders of `Order` class
 - `self.closed_trades` list of closed trades of `Trade` class
 - `self.trades` list of open trades of `Trade` class

`dir_(Position)`
```
['close', 'is_long', 'is_short', 'pl', 'pl_pct', 'size']
```
`dir_(Trade)`
```
['_Trade__set_contingent', '_copy', '_replace', '_sl_order', '_tp_order', 'close', 'entry_bar', 'entry_price', 'entry_time', 'exit_bar', 'exit_price', 'exit_time', 'is_long', 'is_short', 'pl', 'pl_pct', 'size', 'sl', 'tp', 'value']
```
`dir_(Order)`
```
['_replace', 'cancel', 'is_contingent', 'is_long', 'is_short', 'limit', 'parent_trade', 'size', 'sl', 'stop', 'tp']
```

In [35]:
print(dir_(Order))

['_replace', 'cancel', 'is_contingent', 'is_long', 'is_short', 'limit', 'parent_trade', 'size', 'sl', 'stop', 'tp']


In [3]:
class SMA2(Strategy):
    
    n_fast = 3
    n_slow = 10

    def init(self):
        self.n_fast = self.I(SMA, self.data.Close, self.n_fast)
        self.n_slow = self.I(SMA, self.data.Close, self.n_slow)

    def next(self):

        if crossover(self.n_fast, self.n_slow):
            self.buy()
            
        if crossover(self.n_slow, self.n_fast):
            self.sell()
            

bt = Backtest(GAZP, SMA2, commission=.002, exclusive_orders=True)
stats = bt.optimize(n_fast=range(3,10),
                    n_slow=range(5,15),
                    constraint=lambda p: p.n_fast < p.n_slow,
                    maximize="Return (Ann.) [%]"
                   )
stats, stats._strategy

(Start                     2015-12-23 00:00:00
 End                       2020-12-18 00:00:00
 Duration                   1822 days 00:00:00
 Exposure Time [%]                      98.324
 Equity Final [$]                      15460.2
 Equity Peak [$]                       15460.2
 Return [%]                             54.602
 Buy & Hold Return [%]                 57.7827
 Return (Ann.) [%]                     9.15772
 Volatility (Ann.) [%]                 26.3029
 Sharpe Ratio                         0.348164
 Sortino Ratio                        0.618512
 Calmar Ratio                         0.245119
 Max. Drawdown [%]                    -37.3603
 Avg. Drawdown [%]                    -7.39882
 Max. Drawdown Duration      736 days 00:00:00
 Avg. Drawdown Duration       88 days 00:00:00
 # Trades                                  113
 Win Rate [%]                          35.3982
 Best Trade [%]                        41.7828
 Worst Trade [%]                      -7.00464
 Avg. Trade [

In [7]:
stats._strategy.position

<Position: 0 (0 trades)>

# Simple SMA 4 (long/short) strategy 

In [3]:
class SMA4(Strategy):
    
    n_long_fast = 10
    n_long_slow = 100
    n_short_fast = 10
    n_short_slow = 100

    def init(self):
        self.n_long_fast = self.I(SMA, self.data.Close, self.n_long_fast)
        self.n_long_slow = self.I(SMA, self.data.Close, self.n_long_slow)
        self.n_short_fast = self.I(SMA, self.data.Close, self.n_short_fast)
        self.n_short_slow = self.I(SMA, self.data.Close, self.n_short_slow)

    def next(self):

        if crossover(self.n_long_fast, self.n_long_slow):
            self.buy()
            
        if self.position.is_long and crossover(self.n_long_slow, self.n_long_fast):
            self.position.close()
            
        if crossover(self.n_short_slow, self.n_short_fast):
            self.sell()
            
        if self.position.is_short and crossover(self.n_short_fast, self.n_short_slow):
            self.position.close()
            

bt = Backtest(GAZP, SMA4, commission=.002)
stats = bt.optimize(n_long_fast=range(3, 15, 2),
                    n_long_slow=range(10, 50, 5),
                    n_short_fast=range(3, 15, 2),
                    n_short_slow=range(10, 50, 5),
                    maximize="Return (Ann.) [%]",
                    constraint=lambda p: p.n_long_fast < p.n_long_slow 
                    and p.n_short_fast < p.n_short_slow)
stats, stats._strategy

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py:1474: UserWarning: Searching for best of 2116 configurations.
  output = _optimize_grid()


(Start                     2015-12-22 00:00:00
 End                       2020-12-18 00:00:00
 Duration                   1823 days 00:00:00
 Exposure Time [%]                     69.5375
 Equity Final [$]                      18467.1
 Equity Peak [$]                       18467.1
 Return [%]                            84.6714
 Buy & Hold Return [%]                 60.2703
 Return (Ann.) [%]                     13.1188
 Volatility (Ann.) [%]                 23.1877
 Sharpe Ratio                         0.565767
 Sortino Ratio                         1.09321
 Calmar Ratio                         0.402269
 Max. Drawdown [%]                    -32.6121
 Avg. Drawdown [%]                    -6.74366
 Max. Drawdown Duration     1112 days 00:00:00
 Avg. Drawdown Duration       99 days 00:00:00
 # Trades                                  116
 Win Rate [%]                          42.2414
 Best Trade [%]                        39.3383
 Worst Trade [%]                      -7.34764
 Avg. Trade [

# SMA 2 with tp and sl

In [4]:
class SMA2TpSl(Strategy):
    n_fast = 3
    n_slow = 10
    tp_long = 1.05
    sl_long = 0.95
    tp_short = 0.95
    sl_short = 1.05

    def init(self):
        self.n_fast = self.I(SMA, self.data.Close, self.n_fast)
        self.n_slow = self.I(SMA, self.data.Close, self.n_slow)

    def next(self):

        if crossover(self.n_fast, self.n_slow):
            self.buy(sl=self._data.Close[-1] * self.sl_long,
                     tp=self._data.Close[-1] * self.tp_long)

        if crossover(self.n_slow, self.n_fast):
            self.sell(tp=self._data.Close[-1] * self.tp_short,
                      sl=self._data.Close[-1] * self.sl_short)


bt: Backtest = Backtest(GAZP, SMA2TpSl, commission=.002, margin=.5)
s: Union[Series, Tuple[Series, Series], Tuple[Series, Series, dict]] = bt.optimize(
        n_fast=range(2, 5),
        n_slow=range(3, 6),
        tp_long=list(np.arange(1.01,2.0,.1)),
        sl_long=list(np.arange(.8,.95,0.05)),
        tp_short=list(np.arange(.8,.95,0.05)),
        sl_short=list(np.arange(1.01,2.0,.1)),
        constraint=lambda p: p.n_fast < p.n_slow
)
s, s._strategy

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py:1474: UserWarning: Searching for best of 5400 configurations.
  output = _optimize_grid()


(Start                     2015-12-22 00:00:00
 End                       2020-12-18 00:00:00
 Duration                   1823 days 00:00:00
 Exposure Time [%]                     98.8836
 Equity Final [$]                      88413.5
 Equity Peak [$]                       88413.5
 Return [%]                            784.135
 Buy & Hold Return [%]                 60.2703
 Return (Ann.) [%]                     54.9564
 Volatility (Ann.) [%]                 92.8219
 Sharpe Ratio                         0.592062
 Sortino Ratio                         1.54655
 Calmar Ratio                          0.89479
 Max. Drawdown [%]                    -61.4182
 Avg. Drawdown [%]                    -8.85082
 Max. Drawdown Duration      450 days 00:00:00
 Avg. Drawdown Duration       45 days 00:00:00
 # Trades                                   66
 Win Rate [%]                          84.8485
 Best Trade [%]                        21.5042
 Worst Trade [%]                         -13.1
 Avg. Trade [

# SMA 4 with tp and sl

In [8]:
class SMA4TpSl(Strategy):
    
    n_long_fast = 10
    n_long_slow = 100
    n_short_fast = 10
    n_short_slow = 100
    tp_long =1.2
    sl_long = .8
    tp_short = .9
    sl_short = 1.4

    def init(self):
        self.n_long_fast = self.I(SMA, self.data.Close, self.n_long_fast)
        self.n_long_slow = self.I(SMA, self.data.Close, self.n_long_slow)
        self.n_short_fast = self.I(SMA, self.data.Close, self.n_short_fast)
        self.n_short_slow = self.I(SMA, self.data.Close, self.n_short_slow)

    def next(self):

        if crossover(self.n_long_fast, self.n_long_slow):
            self.buy(sl=self._data.Close[-1] * self.sl_long,
                     tp=self._data.Close[-1] * self.tp_long)
            
        if crossover(self.n_short_slow, self.n_short_fast):
            self.sell(tp=self._data.Close[-1] * self.tp_short,
                      sl=self._data.Close[-1] * self.sl_short)

            

bt = Backtest(GAZP, SMA4TpSl, commission=.002)
stats = bt.optimize(n_long_fast=range(3, 15, 2),
                    n_long_slow=range(10, 50, 5),
                    n_short_fast=range(3, 15, 2),
                    n_short_slow=range(10, 50, 5),
                    tp_long=list(np.arange(1.01,1.15,.05)),
                    sl_long=list(np.arange(.6,.9,0.05)),
                    tp_short=list(np.arange(.8,.95,0.05)),
                    sl_short=list(np.arange(1.1,1.3,.05)),
                    maximize="Return (Ann.) [%]",
                    constraint=lambda p: p.n_long_fast < p.n_long_slow 
                    and p.n_short_fast < p.n_short_slow)
stats, stats._strategy

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py:1474: UserWarning: Searching for best of 533232 configurations.
  output = _optimize_grid()


(Start                     2015-12-22 00:00:00
 End                       2020-12-18 00:00:00
 Duration                   1823 days 00:00:00
 Exposure Time [%]                     83.0941
 Equity Final [$]                      55438.5
 Equity Peak [$]                       55438.5
 Return [%]                            454.385
 Buy & Hold Return [%]                 60.2703
 Return (Ann.) [%]                     41.0828
 Volatility (Ann.) [%]                 28.7641
 Sharpe Ratio                          1.42827
 Sortino Ratio                         3.49151
 Calmar Ratio                          2.25569
 Max. Drawdown [%]                     -18.213
 Avg. Drawdown [%]                    -2.90287
 Max. Drawdown Duration      257 days 00:00:00
 Avg. Drawdown Duration       20 days 00:00:00
 # Trades                                   27
 Win Rate [%]                          92.5926
 Best Trade [%]                        12.3951
 Worst Trade [%]                       -10.124
 Avg. Trade [

In [13]:
bt.plot()

In [7]:
class SMA4TpSl(Strategy):
    
    n_long_fast = 9
    n_long_slow = 15
    n_short_fast = 5
    n_short_slow = 15
    tp_long =1.11
    sl_long = .9
    tp_short = .9
    sl_short = 1.15
    size_long = .9
    size_short = .9

    def init(self):
        
        self.n_long_fast = self.I(SMA, self.data.Close, self.n_long_fast)
        self.n_long_slow = self.I(SMA, self.data.Close, self.n_long_slow)
        self.n_short_fast = self.I(SMA, self.data.Close, self.n_short_fast)
        self.n_short_slow = self.I(SMA, self.data.Close, self.n_short_slow)

    def next(self):
        
        if not (self.position.is_long or self.position.is_short):

            if crossover(self.n_long_fast, self.n_long_slow):
                self.buy(sl=self._data.Close[-1] * self.sl_long,
                         tp=self._data.Close[-1] * self.tp_long,
                         size=self.size_long)

            if crossover(self.n_short_slow, self.n_short_fast):
                self.sell(tp=self._data.Close[-1] * self.tp_short,
                          sl=self._data.Close[-1] * self.sl_short,
                          size=self.size_short)

            

bt = Backtest(GAZP, SMA4TpSl, commission=.002, exclusive_orders=False)
stats = bt.optimize(n_long_fast=[2,3,4],
                    n_long_slow=[13,14,15],
                    n_short_fast=[3,4,5],
                    n_short_slow=[13,14,15],
                    tp_long=[1.08,1.1,1.12],
                    sl_long=[.78,.8,.82,],
                    tp_short=[.88,.9,.92],
                    sl_short=[1.13,1.15,1.17],
                    size_long=[.95,.98],
                    size_short=[.95,.98],
                    maximize="Return (Ann.) [%]",
                    constraint=lambda p: p.n_long_fast < p.n_long_slow 
                    and p.n_short_fast < p.n_short_slow)
stats, stats._strategy

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py:1474: UserWarning: Searching for best of 26244 configurations.
  output = _optimize_grid()


(Start                     2015-12-22 00:00:00
 End                       2020-12-18 00:00:00
 Duration                   1823 days 00:00:00
 Exposure Time [%]                     85.5662
 Equity Final [$]                      50496.9
 Equity Peak [$]                       50496.9
 Return [%]                            404.969
 Buy & Hold Return [%]                 60.2703
 Return (Ann.) [%]                     38.4605
 Volatility (Ann.) [%]                 27.6241
 Sharpe Ratio                          1.39228
 Sortino Ratio                         3.31258
 Calmar Ratio                          2.49709
 Max. Drawdown [%]                    -15.4021
 Avg. Drawdown [%]                    -3.39154
 Max. Drawdown Duration      252 days 00:00:00
 Avg. Drawdown Duration       25 days 00:00:00
 # Trades                                   17
 Win Rate [%]                              100
 Best Trade [%]                        13.2275
 Worst Trade [%]                       7.97731
 Avg. Trade [

In [40]:
np.random.seed(42)
returns = np.random.randn(10)
returns.mean()*100

44.80611116987562

In [41]:
win_rate = (returns>0).sum()/len(returns)*100
returns[returns>0].mean()*win_rate + returns[returns<0].mean()*(100-win_rate)

44.80611116987563